In [ ]:
import os
import time

import tfx
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft

from tensorflow_metadata.proto.v0 import schema_pb2

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2
from tfx.components import CsvExampleGen
from tfx.components import SchemaGen
from tfx.components.common_nodes.importer_node import ImporterNode
from tfx.components import ExampleValidator

# Artifact, Serving Model, Data Locations

In [ ]:
ARTIFACT_STORE = os.path.join(os.sep, 'Users', 'CarlosMonsivais', 'Desktop', 'football_manager', 'Arts')
SERVING_MODEL_DIR=os.path.join(os.sep, 'Users', 'CarlosMonsivais', 'Desktop', 'football_manager', 'Mod')
DATA_ROOT = '/Users/CarlosMonsivais/Desktop/football_manager/Data_Dic'

print('Artifact Storage: {}'.format(ARTIFACT_STORE))
print('Serving Model Storage: {}'.format(SERVING_MODEL_DIR))
print('Data Root: {}'.format(DATA_ROOT))

# Pipeline Names

In [ ]:
PIPELINE_NAME = 'tfx-football-pipeline'
PIPELINE_ROOT = os.path.join(ARTIFACT_STORE, PIPELINE_NAME, time.strftime("%Y%m%d_%H%M%S"))

# Creating a directory in the Artifact Storage variable where the pipline metadata will be stored using this timestamp.
os.makedirs(PIPELINE_ROOT, exist_ok = True)

print('Created a directory here:{} \nwhere the pipleine metadata artifacts will be stored.'.format(PIPELINE_ROOT))

# Only for Jupyter Notebook Interactivity

In [ ]:
context = InteractiveContext(pipeline_name = PIPELINE_NAME,
                             pipeline_root = PIPELINE_ROOT,
                             metadata_connection_config = None)

# Reading in Data

In [ ]:
# Doing an 80:20 Train Eval Split
output_config = example_gen_pb2.Output(split_config = example_gen_pb2.SplitConfig(splits=[example_gen_pb2.SplitConfig.Split(name = 'train_data', hash_buckets = 4),
                                                                                          example_gen_pb2.SplitConfig.Split(name = 'test_data', hash_buckets = 1)
                                                                                         ]
                                                                                 )
                                      )

In [ ]:
example_gen = tfx.components.CsvExampleGen(input_base = DATA_ROOT,
                                           output_config = output_config)

In [ ]:
context.run(example_gen)

In [ ]:
# Shows features
examples_uri = example_gen.outputs['examples'].get()[0].uri
tfrecord_filenames = [os.path.join(examples_uri, 'train_data', name)
                      for name in os.listdir(os.path.join(examples_uri, 'train_data'))]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
for tfrecord in dataset.take(2):
  example = tf.train.Example()
  example.ParseFromString(tfrecord.numpy())
  for name, feature in example.features.feature.items():
    if feature.HasField('bytes_list'):
        value = feature.bytes_list.value
    if feature.HasField('float_list'):
        value = feature.float_list.value
    if feature.HasField('int64_list'):
        value = feature.int64_list.value
    print('{}: {}'.format(name, value))
  print('******')

# StaisticsGen Component

In [ ]:
statistics_gen = tfx.components.StatisticsGen(examples = example_gen.outputs['examples'])

In [ ]:
context.run(statistics_gen)

In [ ]:
context.show(statistics_gen.outputs['statistics'])

# SchemaGen

In [ ]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'],
                       infer_feature_shape=False)

In [ ]:
context.run(schema_gen)

In [ ]:
context.show(schema_gen.outputs['schema'])

## Updating SchemaGen File (In case you want to make any changes to the file.)

In [ ]:
# You can chage this file to customize it as you want it for your schema.
schema_proto_path = '{}/{}'.format(schema_gen.outputs['schema'].get()[0].uri, 'schema.pbtxt')
schema = tfdv.load_schema_text(schema_proto_path)

In [ ]:
# tfdv.set_domain(schema, 'Cover_Type', schema_pb2.IntDomain(name='Cover_Type', min=0, max=6, is_categorical=True))   (for categorical variable)
tfdv.set_domain(schema, 'Ability',  schema_pb2.IntDomain(name='Ability', min=0, max=20))

In [ ]:
schema_dir = os.path.join(ARTIFACT_STORE, 'schema')
tf.io.gfile.makedirs(schema_dir)
schema_file = os.path.join(schema_dir, 'schema.pbtxt')

tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

In [ ]:
schema_importer = ImporterNode(instance_name = 'Schema_Importer',
                               source_uri = schema_dir,
                               artifact_type=tfx.types.standard_artifacts.Schema,
                               reimport=False)

In [ ]:
context.run(schema_importer)

In [ ]:
context.show(schema_importer.outputs['result'])

# Example Validator (Anomaly Detection)

In [ ]:
example_validator = ExampleValidator(instance_name = "Data_Validation",
                                     statistics = statistics_gen.outputs['statistics'],
                                     schema = schema_importer.outputs['result'])

In [ ]:
context.run(example_validator)

In [ ]:
train_uri = example_validator.outputs['anomalies'].get()[0].uri
train_anomalies_filename = os.path.join(train_uri, "train/anomalies.pbtxt")

In [ ]:
train_anomalies_filename

In [ ]:
context.show(example_validator.outputs['output'])

# Transform